In [1]:
import warnings
import tensorflow as tf
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')
import numpy as np
import pandas as pd
import yfinance as yf

import sys
sys.path.insert(0, "../Src/")

%autosave 5

Autosaving every 5 seconds


In [2]:
def loadData(ticker):
    train = yf.download(ticker, start="2020-01-01", end="2024-03-01")[['Close','High','Low']].reset_index()
    test = yf.download(ticker, start="2024-03-01", end="2024-10-13")[['Close','High','Low']].reset_index()

    columnRenameDict = {
        'Date' : 'ds',
        'Close': 'y',
        'High': 'cap',
        'Low': 'floor'
    }
    
    train = train.rename(columns = columnRenameDict)
    test = test.rename(columns = columnRenameDict)
    
    return train,test

In [3]:
def processDataForLSTM(data, timeStep=20):
    data = np.array(data)
    X, y = [], []
    for i in range(len(data) - timeStep - 1):
        inputValues = list(np.array(data[i:i + timeStep]).reshape(-1, 1))
        if (i + timeStep) < len(data):
            X.append(inputValues)
            y.append(data[i + timeStep])
        else:
            print(f"Index {i + timeStep} is out of bounds for data length {len(data)}")
    return np.array(X), np.array(y)
    

In [4]:
def main():
    train,test = loadData("MSFT")
    display(train)
    display(test)
    XTrain,yTrain = processDataForLSTM(train['y'])
    XTest,yTest = processDataForLSTM(test['y'])
    print('XTrain.shape: ', XTrain.shape)
    print('yTrain.shape: ', yTrain.shape)
    print('XTest.shape: ', XTest.shape)
    print('yTest.shape: ', yTest.shape)

In [5]:
if __name__ == '__main__':
    main()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,ds,y,cap,floor
0,2020-01-02,160.619995,160.729996,158.330002
1,2020-01-03,158.619995,159.949997,158.059998
2,2020-01-06,159.029999,159.100006,156.509995
3,2020-01-07,157.580002,159.669998,157.320007
4,2020-01-08,160.089996,160.800003,157.949997
...,...,...,...,...
1042,2024-02-23,410.339996,415.859985,408.970001
1043,2024-02-26,407.540009,412.160004,407.359985
1044,2024-02-27,407.480011,408.320007,403.850006
1045,2024-02-28,407.720001,409.299988,405.320007


,ds,y,cap,floor
0,2024-03-01,415.500000,415.869995,410.880005
1,2024-03-04,414.920013,417.350006,412.320007
2,2024-03-05,402.649994,414.250000,400.640015
3,2024-03-06,402.089996,405.160004,398.390015
4,2024-03-07,409.140015,409.779999,402.239990
...,...,...,...,...
151,2024-10-07,409.540009,417.109985,409.000000
152,2024-10-08,414.709991,415.660004,408.170013
153,2024-10-09,417.459991,420.380005,414.299988
154,2024-10-10,415.839996,417.350006,413.149994


XTrain.shape:  (1026, 20, 1)
yTrain.shape:  (1026,)
XTest.shape:  (135, 20, 1)
yTest.shape:  (135,)
